In [1]:
import pandas as pd
import random
import numpy

from bs4 import BeautifulSoup
import requests
import re
import json
import csv

from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select 

from time import sleep

pd.set_option('display.max_columns', None)

C:\Users\Isabel\AppData\Local\Temp\ipykernel_8384\3864169933.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
fallos = []

In [2]:
with open ('../Fase1_datos/lista_drama.json', 'r') as fichero:
    drama = json.load(fichero)

In [11]:
len(drama)

5210

In [29]:
busqueda = drama[1801:2801]

In [30]:
len(busqueda)

1000

In [12]:
info_por_peli = []

In [31]:
len(info_por_peli)

846

In [17]:
len(fallos)

0

In [31]:
# llegar a web:
driver = webdriver.Chrome()
driver.get('https://www.imdb.com/')
driver.maximize_window()
sleep(random.randint(1,3))


# rechazar cookies
driver.find_element('css selector', '#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-hLBbgP.sc-ftTHYK.dcvrLS.dufgkr.ecppKW').click()
sleep(random.randint(1,3))


# variables donde iremos almacenando todo
#info_por_peli = []


# entrando en cada título
for peli in busqueda:
    lista_info = [f'{peli[0]}',]

    try:
    # metemos el ID en el buscador
        try:
            driver.find_element('css selector', '#suggestion-search').send_keys(peli[0], Keys.ENTER)
            sleep(random.randint(1,3))

            # extraemos info
            info = driver.find_element('xpath' , f'//*[@id="__next"]/main/div/section[1]/section/div[3]/section').text
            lista_info.append(info.split('\n'))

            info_por_peli.append(lista_info)
        except:
            driver.find_element('css selector', '#error > div.error_attrib > a').click()
            
    except:
        fallos.append(peli)


